In [1]:
import ujson
import gzip
import numpy as np
import math
from os.path import isfile, isdir
from struct import pack, unpack
from os import makedirs
import os
from intbitset import intbitset
from struct import pack
import itertools
import scipy.sparse, scipy.io, scipy.stats
import intbitset
import pickle as pickle

First, we need to download the actual VK user data and adjacency matrix. First, the users data (770Mb) [from GoogleDrive](https://drive.google.com/file/d/0B2i787wUBKkgVC1XcDBic21FaFk/view?usp=sharing)

I also provide small adjacency matrix for first 1M VK users (123Mb). You can download it [from GoogleDrive](https://drive.google.com/file/d/0B2i787wUBKkgTW43cGUzR1RCRFU/view?usp=sharing). If someome needs full adjacency matrix, I can also provide it.

In [182]:
maxid = 1000000

In [2]:
data = np.load('D:\\vk\\userinfo.npz') # Location of the user info file
countryids = data['countryids']
#cityids = data['cityids'] # we will use only country IDs in this notebook to sustain the memory constraint lower than 16Gb
#uniids = data['uniids']
#facids = data['facids']
del(data)

In [4]:
np.histogram(countryids, range=(0, 500))

(array([293282901,   3031146,    765182,   1009068,   1908958,         2,
                3,         2,         2,         1]),
 array([   0.,   50.,  100.,  150.,  200.,  250.,  300.,  350.,  400.,
         450.,  500.]))

In [5]:
np.clip(countryids, 0, 235, out=countryids) # Everything except the first 235 IDs is garbage, remove it

array([ 0.,  1.,  1., ...,  0.,  0.,  0.])

In [7]:
def save_sparse_csr(filename,array):
    np.savez_compressed(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

def load_sparse_csr(filename):
    loader = np.load(filename)
    return scipy.sparse.csr_matrix((  loader['data'], loader['indices'], loader['indptr']),
                         shape = loader['shape'])

In [8]:
adj = load_sparse_csr('D:\\vk\\adj_small.npz')

In [19]:
distmatrix = np.zeros((236,236), dtype=np.float)

In [20]:
nnz = adj.nonzero()
for row, col in zip(nnz[0], nnz[1]):
    c_r = countryids[row]
    c_c = countryids[col]
    distmatrix[c_c, c_r] += 1
    distmatrix[c_r, c_c] += 1    
del(nnz)

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:5: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Anaconda\lib\site-packages\ipykernel\__main__.py:6: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [337]:
D = distmatrix.copy()
D = np.log(D+1)
D /= np.sum(D, axis=0) + 1
D = D[1:,1:] + 10*np.eye(235) # dirty

In [338]:
plt.imshow(D)
plt.show()

In [183]:
x0 = scipy.sparse.dok_matrix((maxid, 236), dtype=np.float)

In [184]:
x0[np.arange(maxid), countryids[:maxid]] = 1

In [185]:
x0 = x0[:, 1:]
x0 = x0.tocsr()

In [355]:
xi = scipy.sparse.lil_matrix(x0, copy=True)

In [356]:
for epoch in range(4):
    err = 0
    for uid in np.arange(maxid):
        uids = adj[uid].nonzero()[1]
        if len(uids) == 0:
            continue
        arr = xi[uids].dot(D).sum(axis=0) #remove .dot(D) for faster simplified version

        if np.max(arr) == 0:
            continue
        if np.isnan(np.min(arr)):
            continue
        arr /= np.sum(arr)
        arr += x0[uid]
        arr /= np.sum(arr)
        arr = scipy.stats.threshold(arr, threshmin=0.01)
        arr /= np.sum(arr)
        err += np.linalg.norm(xi[uid]-arr)
        xi[uid] = arr
        if uid % 10000 == 0:
            print uid, err/(uid+1)
    print 'Total error=',err
    save_sparse_csr('D:\\vk\\dcountryinfo\\ux'+str(epoch+1)+'.npz', xi.tocsr())

10000 0.054393511261
20000 0.053171648398
50000 0.0506615454924
60000 0.0502575458566
70000 0.049985515676
80000 0.0497350253726
90000 0.0491094307829
100000 0.0490249448211
110000 0.0486061992423
130000 0.0479323235347
140000 0.0475941657046
150000 0.0472791914364
200000 0.0466113621149
220000 0.0463916382837
240000 0.0461324500008
250000 0.0459917842569
270000 0.0456883502094
280000 0.0454711080492
300000 0.0451725447743
320000 0.0448420455162
330000 0.0448255374088
340000 0.0446994277234
350000 0.044661991693
360000 0.0445182256725
370000 0.0444964017392
380000 0.0444102716889
400000 0.044314737987
410000 0.0442112564735
420000 0.044144244453
430000 0.0440707589213
440000 0.0439645145594
450000 0.043987981396
460000 0.0439331838866
480000 0.0437729375202
490000 0.0437512045592
500000 0.0436718856144
510000 0.0436348832393
530000 0.0434957435834
540000 0.0434388525044
550000 0.043428100383
570000 0.0434271851817
580000 0.0433793254055
590000 0.0433447166914
600000 0.0433103375723
620

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:14: DeprecationWarning: Implicitly casting between incompatible kinds. In a future numpy release, this will raise an error. Use casting="unsafe" if this is intentional.
